# 1. SVM (6k)

## 1.1. Revisión de la base de datos

Con la BBDD que me compartió Paúl, voy a crear un modelo que, a partir de un indicador de calidad.

In [1]:
import pandas as pd
import numpy as np
import pyreadstat

Importamos el archivo, revisamos las columnas, nos quedamos sólo con las que nos interesan y vemos cuántas filas tenemos.

In [2]:
h=pd.read_spss('rawdata/BDD 10958 EPC and NO EPC - PEZ_2.sav')

In [3]:
print(h.columns.tolist())

['OBJECTID', 'codigo_inmueble1', 'Title', 'Type_build', 'Type_opera', 'Link', 'Location', 'Lat_X', 'Lon_Y', 'Climatic_Z', 'Nom_Mun', 'precio_eur', 'superficie', 'superficie2', 'Unit_price', 'Ln_total_pr', 'Ln_unit_pr', 'numero_habitaciones', 'numero_bano', 'ratio_bano_hab', 'numero_aseo', 'ascensor', 'interac_planta', 'numero_de_piso', 'anyo_constr_ponderad', 'antig_ponderad', 'Inverse_Age', 'Year_Before_1981', 'Year_1982_2006', 'Year_After_2007', 'superficie_terraza_m2', 'grand_terr_20m2', 'superficie_jardin_m2', 'superficie_salon', 'bool_despacho', 'bool_buhardilla', 'bool_trastero', 'bool_lavadero', 'bool_piscina_comunitaria', 'bool_jardin_comunitario', 'bool_amueblado', 'bool_ascensor', 'descripcion', 'bool_aire_acondicionado', 'bool_calefaccion', 'bool_chimenea', 'texto_destacado', 'Description', 'calificacion_consumo_letra', 'calificacion_consumo_valor', 'calificacion_emision_letra', 'calificacion_emision_valor', 'Dum_EPC', 'EPC_A_emision', 'EPC_B_emision', 'EPC_C_emision', 'EPC_

Voy a buscar cuáles son dummies. De acuerdo con el diccionario, *alta_calidad* es la que sintetiza la calidad del inmueble. Con ella trabajaré.

In [4]:
unique_counts = h.nunique()
categoricas = unique_counts[unique_counts < 5].index
categoricas

Index(['Type_build', 'Type_opera', 'numero_aseo', 'ascensor',
       'Year_Before_1981', 'Year_1982_2006', 'Year_After_2007',
       'grand_terr_20m2', 'bool_despacho', 'bool_buhardilla', 'bool_trastero',
       'bool_lavadero', 'bool_piscina_comunitaria', 'bool_jardin_comunitario',
       'bool_amueblado', 'bool_ascensor', 'bool_aire_acondicionado',
       'bool_calefaccion', 'bool_chimenea', 'Dum_EPC', 'EPC_A_emision',
       'EPC_B_emision', 'EPC_C_emision', 'EPC_D_emision', 'EPC_E_emision',
       'EPC_F_emision', 'EPC_G_emision', 'dum_acces_viappal', 'calidad_cocina',
       'diseny_cocina', 'alta_calidad', 'reform_inmob', 'dum_mar_200m',
       'dum_ttpp_riel_urb', 'C_contempo', 'C_estado', 'C_armarios',
       'B_contempo', 'B_estado', 'B_lavamano', 'R_contempo', 'R_estado',
       'R_carpinte', 'R_singular', 'R_ventana', 'Dum_precio', 'scrap_year',
       'Y_2023', 'Filtro', 'filter_$', 'EPC_AB', 'EPC_ABC', 'QCL_1',
       'EPC_A_emission_2023', 'EPC_B_emission_2023', 'EPC_C_em

Ahora voy a buscar qué columnas tienen valores *string*, es decir, letras.

In [5]:
colstring = h.select_dtypes(include=['object'])
print(colstring.columns)

Index(['Title', 'Type_build', 'Type_opera', 'Link', 'Location', 'Climatic_Z',
       'Nom_Mun', 'descripcion', 'texto_destacado', 'Description',
       'calificacion_consumo_letra', 'calificacion_emision_letra', 'persona',
       'AUTOP_NEAR'],
      dtype='object')


Reviso a mayor profundidad qué información tienen aquellas que, creo, pueden acercarme a saber el anuncio de venta.

In [6]:
colstring[['texto_destacado','Description']]

,texto_destacado,Description
0,4 habitaciones en 11 de setembre,"Piso reformado de 4 habitaciones, salón comedo..."
1,PIS BENET MATEU/ MANUEL DE FALLA,"BENET MATEU, PIS D´ORIGEN AMB MOLT BONA DISTRI..."
2,Apartamento tipo casa Canyelles,Apartamento pero con acceso independiente desd...
3,TODO EXTERIOR Y REFORMADO,"[A2977]PISAZO, EL MEJOR DE LA ZONA.FENOMENAL P..."
4,102 M2 EXTERIORES CON ASCENSOR,[A3001]VIVIENDA EN LA CALLE GARROFER DE SANT I...
...,...,...
5462,Piso en venta en Can Pantiquet-Riera Seca,Piso En Mollet Del Vallès!Ubicado a 600m de la...
5463,BUENA UBICACIÓN!,"La Casa Agency presenta en Exclusividad, esta ..."
5464,PISO EN PLANTA BAJA CON PATIO DE 60m²,Mis Finques promociona esta planta baja con pa...
5465,PISO CON TERRAZA,PISO CON TERRAZAPiso con TERRAZA DE 40M2. La v...


Ahora que ya sé qué columnas me interesan, las selecciono y empiezo a trabajar.

In [7]:
hab=h[['Description','alta_calidad']]
hab.info()
hab.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5467 entries, 0 to 5466
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Description   5467 non-null   object 
 1   alta_calidad  5467 non-null   float64
dtypes: float64(1), object(1)
memory usage: 85.6+ KB


,Description,alta_calidad
0,"Piso reformado de 4 habitaciones, salón comedo...",0.0
1,"BENET MATEU, PIS D´ORIGEN AMB MOLT BONA DISTRI...",0.0
2,Apartamento pero con acceso independiente desd...,0.0
3,"[A2977]PISAZO, EL MEJOR DE LA ZONA.FENOMENAL P...",1.0
4,[A3001]VIVIENDA EN LA CALLE GARROFER DE SANT I...,1.0


Una de las cosas a las que pueden ser sensibles este tipo de modelos es al desequilibrio entre las clases (por ejemplo, hay muchas más instancias de una clase que de la otra), de manera que su precisión puede ser alta incluso si el modelo predice incorrectamente la clase menos frecuente con frecuencia. Veamos cuántos 0 y 1 tenemos en la columna **alta_calidad**.

In [8]:
conteo = hab['alta_calidad'].value_counts()
print(conteo)

alta_calidad
0.0    4458
1.0    1009
Name: count, dtype: int64


Vamos a tener que cuidar cómo se re-distribuyen estos valores cuando elija datos de entrenamiento y testeo.

## 1.2. Elaboración del modelo

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import nltk
from nltk.stem import WordNetLemmatizer
import unicodedata

# Descargar recursos necesarios para NLTK
nltk.download('punkt')
nltk.download('wordnet')

# Definir función para lematización
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    tokens = nltk.word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

# Definir función para eliminar tildes y convertir a minúsculas
def preprocess_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = ''.join(char for char in unicodedata.normalize('NFD', text) if unicodedata.category(char) != 'Mn')  # Eliminar tildes
    return text

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(hab['Description'], hab['alta_calidad'], 
                                                    test_size=0.2, 
                                                    random_state=42) #Ojo: elijo 80-20.

# Preprocesamiento y representación de texto
custom_stopwords = ['de', 'la', 'el', 'los', 'las', 'en', 'para', 'por',
                    'con', 'y', 'o', 'un', 'una', 'que', 'se', 'su', 'sus']  # Agrega más palabras si es necesario
tfidf_vectorizer = TfidfVectorizer(stop_words=custom_stopwords)

# Lematizar y preprocesar el texto
X_train_lemmatized = X_train.apply(lemmatize_text)
X_test_preprocessed = X_test.apply(preprocess_text)
X_test_lemmatized = X_test_preprocessed.apply(lemmatize_text)

# Entrenar el vectorizador TF-IDF
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_lemmatized)
X_test_tfidf = tfidf_vectorizer.transform(X_test_lemmatized)

# Obtener nombres de características y mapearlos a sus índices
feature_names = tfidf_vectorizer.get_feature_names_out()
feature_index_map = {word: idx for idx, word in enumerate(feature_names)}

# Definir diccionario de palabras y factores de multiplicación de peso (eliminando tildes)
word_weight_dict = {'noble': 1.5, 'reformado': 1.5, 'rehabilitado': 1.5, 'amplio': 1.5, 
                    'equipado': 1.5, 'restaurada': 1.5, 'isla': 1.5, 'isleta': 1.5, 'estrenar': 1.5,
                    'conservado': 1.5, 'moderna': 1.5, 'acondicion': 1.5, 'lujo': 1.5, 'muy': 1.5,
                    'diseno': 1.5, 'estado': 1.5, 'grand': 1.5, 'excelente': 1.5, 'vitroceramica': 1.5,
                    'renovado': 1.5, 'impecable': 1.5, 'nuevo': 1.5, 'perfecto': 1.5, 'totalmente': 1.5,
                    'total': 1.5, 'full': 1.5, 'espectacular': 1.5, 'maravillosa': 1.5, 'estupendo': 1.5,
                    'optimo': 1.5, 'magnifica': 1.5, 'ideal': 1.5, 'fantastica': 1.5, 'vanguardia': 1.5,
                    'office': 1.5, 'americana': 1.5, 'abierto': 1.5, 'vistas': 1.5, 'luminoso': 1.5,
                    'iluminada': 1.5, 'bien': 1.5, 'entrar': 1.5, 'actualizado': 1.5, 'conservacion': 1.5,
                    'buen': 1.5, 'condicion': 1.5, 'mantenida': 1.5, 'cuidado': 1.5, 'precioso': 1.5,
                    'bonita': 1.5, 'encanto': 1.5, 'magnifica/co': 1.5, 'senoral': 1.5, 'vista': 1.5,
                    'panoramico': 1.5, 'exterior': 1.5, 'modernista': 1.5, 'acogedora': 1.5, 'regia': 1.5,
                    'gusto': 1.5, 'noucentista': 1.5, 'exclusivo': 1.5, 'neo-clasica': 1.5,
                    'neoclasica': 1.5, 'inmejorable': 1.5, 'fabuloso': 1.5, 'majestuosa': 1.5,
                    'alta calidad': 1.5, 'alto standing': 1.5, 'super': 1.5, 'impresionante': 1.5,
                    'elegante': 1.5, 'esplendido': 1.5, 'calida': 1.5, 'especial': 1.5}

# Modificar pesos según el diccionario
for word, weight_factor in word_weight_dict.items():
    word_no_accents = preprocess_text(word)
    if word_no_accents in feature_index_map:
        word_index = feature_index_map[word_no_accents]
        X_train_tfidf[:, word_index] *= weight_factor
        X_test_tfidf[:, word_index] *= weight_factor

# Entrenar un modelo de clasificación (por ejemplo, SVM)
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_tfidf, y_train)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rojas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rojas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


SVC(kernel='linear')

In [10]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

# Predecir sobre los datos de prueba
y_predSVM = svm_model.predict(X_test_tfidf)

# Evaluar el modelo
accuracySVM = accuracy_score(y_test, y_predSVM)

# Calcular precision, recall y F1 sobre los datos de prueba
precisionSVM = precision_score(y_test, y_predSVM)
recallSVM = recall_score(y_test, y_predSVM)
f1SVM = f1_score(y_test, y_predSVM)
conf_matrixSVM = confusion_matrix(y_test, y_predSVM)

# Imprimir las métricas
print("Precision:", precisionSVM)
print("Recall:", recallSVM)
print("F1 Score:", f1SVM)
print("Exactitud del modelo:", accuracySVM)
print("Confusion Matrix:\n", conf_matrixSVM)

Precision: 0.6666666666666666
Recall: 0.05102040816326531
F1 Score: 0.0947867298578199
Exactitud del modelo: 0.8254113345521024
Confusion Matrix:
 [[893   5]
 [186  10]]


Algunos apuntes: 

- Precisión (Precision): La precisión se refiere a la proporción de las instancias clasificadas como positivas que son realmente positivas.Se calcula como el número de verdaderos positivos dividido por la suma de verdaderos positivos y falsos positivos. **Es útil cuando el costo de los falsos positivos es alto y deseas minimizarlos**.

- Exactitud (Accuracy): La exactitud es la proporción de todas las predicciones que son correctas. Se calcula como la suma de verdaderos positivos y verdaderos negativos dividido por el total de instancias. Es una medida global del rendimiento del modelo y **es útil cuando todas las clases tienen una importancia similar**.

Guardamos.

In [11]:
import joblib

# Guardar el modelo SVM entrenado
joblib.dump(svm_model, 'modelos/SVM2.pkl')

# Guardar el vectorizador TF-IDF entrenado
joblib.dump(tfidf_vectorizer, 'modelos/SVM2-tfidf_vectorizer.pkl')

['modelos/SVM2-tfidf_vectorizer.pkl']

# 2. Redes neuronales (6k)

Esta vez usaremos una red neuronal con tres capas densas, cada una seguida de una capa de abandono (dropout) para evitar el sobreajuste. Utilizamos la función de activación 'relu' en las capas ocultas y 'sigmoid' en la capa de salida para problemas de clasificación binaria. El optimizador Adam se utiliza para minimizar la pérdida de entropía cruzada binaria, y la exactitud (*accuracy*) se utiliza como métrica de evaluación.

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

##### Esto ya lo tengo del modelo anterior, por eso no lo ejecuto
# Dividir los datos en características (X) y etiquetas (y)
#X = habit['texto_destacado']
#y = habit['alta_calidad']

# Dividir los datos en conjuntos de entrenamiento y prueba
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Representación TF-IDF de las características de texto
#tfidf_vectorizer = TfidfVectorizer(stop_words=custom_stopwords)
#X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
#X_test_tfidf = tfidf_vectorizer.transform(X_test)

##### A partir de aquí ya son cosas nuevas para este modelo de redes neuronales
# Escalar las características numéricas
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_tfidf.toarray())
X_test_scaled = scaler.transform(X_test_tfidf.toarray())

# Crear el modelo de redes neuronales
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid') #Sigmoid es más adecuado para variables binarias
])

# Compilar el modelo
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
historyANN = model.fit(X_train_scaled, y_train, #Son las mismas del modelo anterior
                    epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test),
                    verbose=False)

Encuentro los parámetros para comparar con modelo SVM.

In [13]:
# Calcular las predicciones binarias del modelo en el conjunto de datos de prueba
umbral = 0.5  # Umbral para convertir las probabilidades en etiquetas binarias
y_pred_binario = [1 if pred >= umbral else 0 for pred in model.predict(X_test_scaled)]

# Calcular las métricas de evaluación
accuracyANN = accuracy_score(y_test, y_pred_binario)
recallANN = recall_score(y_test, y_pred_binario)
precisionANN = precision_score(y_test, y_pred_binario)
f1ANN = f1_score(y_test, y_pred_binario)
conf_matrixANN = confusion_matrix(y_test, y_pred_binario)

# Imprimir las métricas
print("Accuracy:", accuracyANN)
print("Recall:", recallANN)
print("Precision:", precisionANN)
print("F1 Score:", f1ANN)
print("Confusion Matrix:\n", conf_matrixANN)

35/35 [==============================] - 0s 2ms/step
Accuracy: 0.8126142595978062
Recall: 0.061224489795918366
Precision: 0.36363636363636365
F1 Score: 0.10480349344978165
Confusion Matrix:
 [[877  21]
 [184  12]]


In [14]:
import pickle

# Guardar el modelo
model.save("modelos/NN2-BCN6K")

# Guardar el historial
with open("modelos/NN2-BCN6k/history.pkl", "wb") as f:
    pickle.dump(historyANN.history, f)

INFO:tensorflow:Assets written to: modelos/NN2-BCN6K\assets


INFO:tensorflow:Assets written to: modelos/NN2-BCN6K\assets


In [15]:

# Preprocesamiento del texto en 'texto_destacado' utilizando el mismo TfidfVectorizer y StandardScaler
#X_habit6_tfidf = tfidf_vectorizer.transform(habit6['texto_destacado']) #### Esto ya no lo ejecuto porque ya lo hice con el modelo anterior
X_habit6_scaled = scaler.transform(X_habit6_tfidf.toarray())

#### A partir de aquí ya es nuevo
# Predecir la calidad de los inmuebles en 'habit6' utilizando el modelo de redes neuronales ya entrenado
y_habit6_pred = model.predict(X_habit6_scaled)

# Convertir las predicciones a etiquetas binarias (0 o 1) utilizando un umbral (por ejemplo, 0.5)
umbral = 0.5
acp_LLM_ANN = [1 if pred >= umbral else 0 for pred in y_habit6_pred] #acp de alta_calidad_predicha

# Agregar una columna 'alta_calidad_predicha' al DataFrame 'habit6' con las predicciones del modelo
habit6['acp-LLM-ANN'] = acp_LLM_ANN

#coincidencias6 = (habit6['alta_calidad'] == habit6['acp-LLM-ANN']).sum()

print("Cantidad de valores coincidentes:", (habit6['alta_calidad'] == habit6['acp-LLM-ANN']).sum())


NameError: name 'X_habit6_tfidf' is not defined

In [ ]:
4869/5467

Parece tener un rendimiento ligeramente menor que el **modelo SVM**. Guardamos los resultados.

In [ ]:
# Guardar el modelo entrenado
model.save('modelos/LLM1-ANN')

# Guardar el historial de entrenamiento
dump(history, 'modelos/LLM1-ANN/history.joblib')

## 3. Random Forest (6k)

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

#### Esto ya lo tengo

# Dividir los datos en características (X) y etiquetas (y)
#X = habit['texto_destacado']
#y = habit['alta_calidad']

# Dividir los datos en conjuntos de entrenamiento y prueba
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Representación TF-IDF de las características de texto
#tfidf_vectorizer = TfidfVectorizer(stop_words=custom_stopwords)
#X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
#X_test_tfidf = tfidf_vectorizer.transform(X_test)

#### Esto ya es nuevo
# Crear y entrenar el modelo de Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train_tfidf, y_train)

# Predecir las etiquetas en el conjunto de prueba
y_predRF = random_forest.predict(X_test_tfidf)

# Calcular las métricas de evaluación
accuracyRF = accuracy_score(y_test, y_predRF)
recallRF = recall_score(y_test, y_predRF)
precisionRF = precision_score(y_test, y_predRF)
f1RF = f1_score(y_test, y_predRF)
conf_matrixRF = confusion_matrix(y_test, y_predRF)

# Imprimir las métricas para comparar con SVM y ANN
print("Accuracy:", accuracyRF)
print("Recall:", recallRF)
print("Precision:", precisionRF)
print("F1 Score:", f1RF)
print("Confusion Matrix:\n", conf_matrixRF)

Accuracy: 0.823583180987203
Recall: 0.04081632653061224
Precision: 0.6153846153846154
F1 Score: 0.07655502392344496
Confusion Matrix:
 [[893   5]
 [188   8]]


In [17]:
# Guardar el modelo de Random Forest
joblib.dump(random_forest, 'modelos/RF2-BCN6K.pkl')

['modelos/RF2-BCN6K.pkl']

# 4. Comparamos los resultados de los modelos

In [21]:
# Crear un diccionario con los datos
data = {
    'Modelo': ['SVM', 'ANN', 'RF'],
    'Accuracy': [accuracySVM, accuracyANN, accuracyRF],
    'Precision': [precisionSVM, precisionANN, precisionRF],
    'Recall': [recallSVM, recallANN, recallRF],
    'F1 Score': [f1SVM, f1ANN, f1RF]
}

# Crear un DataFrame con los datos
comparacion = pd.DataFrame(data)

# Mostrar la tabla
print(comparacion)

  Modelo  Accuracy  Precision    Recall  F1 Score
0    SVM  0.825411   0.666667  0.051020  0.094787
1    ANN  0.812614   0.363636  0.061224  0.104803
2     RF  0.823583   0.615385  0.040816  0.076555


- Precisión (Accuracy):
    - La precisión es la proporción de predicciones correctas sobre el total de predicciones.
    - Una precisión del 1.0 indica que todas las predicciones son correctas, mientras que una precisión del 0.0 indica que ninguna predicción es correcta.
    - Es una métrica general del rendimiento del modelo, pero puede ser engañosa si hay un desequilibrio en las clases objetivo.

- Recall (Exhaustividad):
    - La exhaustividad es la proporción de positivos reales que se identificaron correctamente.
    - Una exhaustividad del 1.0 indica que todas las instancias positivas se han identificado correctamente, mientras que una exhaustividad del 0.0 indica que ninguna instancia positiva se ha identificado correctamente.
    - Es útil cuando la identificación de instancias positivas es crítica y no se pueden permitir falsos negativos.

- Precisión (Precision):
    - La precisión es la proporción de instancias positivas predichas que fueron correctamente identificadas.
    - Una precisión del 1.0 indica que todas las instancias predichas como positivas son verdaderas positivas, mientras que una precisión del 0.0 indica que ninguna instancia predicha como positiva es realmente positiva.
    - Es útil cuando es importante evitar falsos positivos.

- F1 Score:
    - El puntaje F1 es la media armónica de precisión y exhaustividad.
    - Proporciona un equilibrio entre precisión y exhaustividad, lo que lo hace útil cuando se desea tener un buen rendimiento en ambas métricas.
    - Un puntaje F1 del 1.0 indica un equilibrio perfecto entre precisión y exhaustividad.

- Matriz de Confusión:
    - La matriz de confusión es una tabla que describe la calidad de las predicciones del modelo.
    - Proporciona una descripción detallada de los resultados de clasificación, mostrando el número de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos.
    - Es útil para identificar dónde el modelo está cometiendo errores y para evaluar el desempeño en cada clase por separado.